# This is a Capstone project for Coursera couse "Applied Data Science Capstone" 


## Objectives - Part 3 - Analysis base on merge data (from wikipedia and geocode data and venues data get from Foursquare api) 

- Show data on map
- Show data on map with clusters 
- Show each cluster's data in tables
        

## Below is data scraping from Wikipedia

In [6]:
import pandas as pd

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np


# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # plotting library


In [7]:
#Read data from Wikipedia using pandas to read it to a dataframe

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

dfs[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
#Find out if there are rows that has Borough is not "Not assigned" however Neighbourhood is "Not assigned"
df_to_exclude = dfs[0].loc[(dfs[0]['Borough'] != 'Not assigned') & (dfs[0]['Neighbourhood'] == 'Not assigned')]
df_to_exclude.shape


(0, 3)

In [9]:
#Base on result of query above, it is clear that there is no data which has 
# Neighbourhood = Not assigned however Borough is different from "Not assigned"

#We can clean data just by removing rows that has Borough is "Not assigned"
df_toronto = dfs[0].loc[dfs[0]['Borough'] != 'Not assigned']
df_toronto

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
#Join rows that has same Postal code and Borough to remove duplications if any
df_toronto_2 = df_toronto.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
#Sort by Postal Code
df_toronto_2.sort_values(by=['Postal Code'])
df_toronto_2

#Seeing the result below we can also conclude that there is no duplications in Postal code and Borough

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [11]:
df_toronto_2.shape

(103, 3)

## Read Geocode data from CSV and Join with data get from Wikipedia to form a dataset with Geocode

In [12]:
#Read the geocodeing of Postalcode from csv file
url_geo = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url_geo)

df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## Below is the data set that is merged

In [13]:
# Joins/merges two dataset base on the Postal code

df_toronto_with_geo = pd.merge(df_toronto_2, df_geo, on=['Postal Code'])
df_toronto_with_geo

df_toronto_with_geo

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [14]:
df_toronto_with_geo.shape

(103, 5)

## Below is analysis part
### Find the center coordinates of Toronto 

In [15]:
address = 'Toronto, NY'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 41.6302032, -74.85848290763056.


### Show map of toronto 

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_with_geo['Latitude'], df_toronto_with_geo['Longitude'], df_toronto_with_geo['Borough'], df_toronto_with_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Focus on one Borough - for example Scarborough
### Get the data set of Scarborough only and load it on map


In [17]:
scarborough_data = df_toronto_with_geo[df_toronto_with_geo['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location_sb = geolocator.geocode(address)
latitude_sb = location_sb.latitude
longitude_sb = location_sb.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 41.6302032, -74.85848290763056.


## Map of Scarborough 
## add marker for each postal code , aka. Borough location. Click on marker will show neighbourhood of the Postal code

In [19]:
# create map of Toronto using latitude and longitude values
map_sb = folium.Map(location=[latitude_sb, longitude_sb], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sb)  
    
map_sb

## Access to Foursquare 

In [20]:
CLIENT_ID = 'B3JXR2BRDOV3HTRROWE405OQCF5ZV54QVPWXPXNZREPFJAOR' # your Foursquare ID
CLIENT_SECRET = 'LH0K4GEDVHCDMOLYXK20EKO3UV0L0DKX45SOTW5AQDJWWQJC' # your Foursquare Secret
ACCESS_TOKEN = 'J2MDKH2UZA2M1KJUI5QXVOZZLJJ12X4R20EDRB0XZWBZQKDD' # your FourSquare Access Token
#{"access_token":"J2MDKH2UZA2M1KJUI5QXVOZZLJJ12X4R20EDRB0XZWBZQKDD"}
VERSION = '20180605'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: B3JXR2BRDOV3HTRROWE405OQCF5ZV54QVPWXPXNZREPFJAOR
CLIENT_SECRET:LH0K4GEDVHCDMOLYXK20EKO3UV0L0DKX45SOTW5AQDJWWQJC


In [21]:
scarborough_data.loc[1, 'Neighbourhood']

'Rouge Hill, Port Union, Highland Creek'

In [22]:
neighborhood_latitude = scarborough_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge Hill, Port Union, Highland Creek are 43.7845351, -79.1604971.


### Now let's get top 100 venues of Malvern, Rouge within radius of 500

In [23]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=B3JXR2BRDOV3HTRROWE405OQCF5ZV54QVPWXPXNZREPFJAOR&client_secret=LH0K4GEDVHCDMOLYXK20EKO3UV0L0DKX45SOTW5AQDJWWQJC&v=20180605&ll=43.7845351,-79.1604971&radius=500&limit=100'

#### Send the get requests and examine the results

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6016e86bc395a9504b3d072d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7890351045, 'lng': -79.15427558741126},
   'sw': {'lat': 43.7800350955, 'lng': -79.16671861258874}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c23d3aaf7ced13a5ed7216d',
       'name': 'Royal Canadian Legion',
       'location': {'address': '45 Lawson Rd',
        'lat': 43.78253332838298,
        'lng': -79.16308473261682,
        'labeledLatLngs': [{'label': 'display',
          'lat': 4

#### Define the get_category_type function 


In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.


In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Royal Canadian Legion,Bar,43.782533,-79.163085
1,Scarborough Historical Society,History Museum,43.788755,-79.162438


Hum, there doesn't seem a lot of venue in this area

## Explore neighborhood in Scarborough

Let's create a function to repeat the same process to all the neighborhoods in Scarborough


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [29]:
print(scarborough_venues.shape)
scarborough_venues.head()

(93, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### Let's check how many venues were returned for each neighborhood

In [30]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",6,6,6,6,6,6


Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


## Analyse Scarborough neighbourhood

In [32]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [33]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.0,...,0.000000,0.083333,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.083333,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.2,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.000000,0.0,...,0.000000,0.125000,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [34]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3         Chinese Restaurant  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4              Pet Store  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1       Thai Restaurant  0.12
2                Bakery  0.12
3                  Bank  0.12
4    Athletics & Sports  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0           Pizza Place  0.17
1    Italian Restaurant  0.08
2  Fast Food Restaurant  0.08
3   Fried Chicken Joint  0.08
4              Pharmacy  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1 

### Put the results above to a pandas frame

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Chinese Restaurant,American Restaurant,Pet Store,Korean BBQ Restaurant,Light Rail Station,Medical Center,Metro Station
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
2,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Gas Station,American Restaurant,Motel
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Italian Restaurant,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Chinese Restaurant,Gas Station,Rental Car Location,Noodle House,Bank
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Park,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center


## 4. Cluster neighbourhoods 

In [58]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
scarborough_merged = scarborough_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head(100) # check the last columns!


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,History Museum,Bar,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Bank,Breakfast Spot,Medical Center,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Noodle House,Korean BBQ Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean BBQ Restaurant,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Gas Station,American Restaurant,Motel
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Convenience Store,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Park,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Latin American Restaurant,Light Rail Station
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,1.0,American Restaurant,Skating Rink,Motel,Park,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,College Stadium,General Entertainment,Skating Rink,Café,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station


In [71]:
scarborough_merged.dropna(inplace=True)
scarborough_merged


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,History Museum,Bar,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Bank,Breakfast Spot,Medical Center,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Noodle House,Korean BBQ Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean BBQ Restaurant,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Gas Station,American Restaurant,Motel
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Convenience Store,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Park,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Latin American Restaurant,Light Rail Station
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,1.0,American Restaurant,Skating Rink,Motel,Park,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,College Stadium,General Entertainment,Skating Rink,Café,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station


In [87]:
# convert the type of Cluster label to interger for using in later dataset merging
scarborough_merged[['Cluster Labels']] = scarborough_merged[['Cluster Labels']].astype(int)
scarborough_merged.dtypes

Postal Code                object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

### Finally, let's visualize the resulting clusters

In [74]:
# create map
map_clusters = folium.Map(location=[latitude_sb, longitude_sb], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

### Cluster 1 

In [78]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,History Museum,Bar,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station


### Cluster 2


In [79]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Intersection,Bank,Breakfast Spot,Medical Center,Restaurant,Rental Car Location,Mexican Restaurant,Electronics Store,Noodle House,Korean BBQ Restaurant
4,Scarborough,1,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Gas Station,American Restaurant,Motel
6,Scarborough,1,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
7,Scarborough,1,Bakery,Bus Line,Intersection,Park,Ice Cream Shop,Soccer Field,Bus Station,Metro Station,Latin American Restaurant,Light Rail Station
8,Scarborough,1,American Restaurant,Skating Rink,Motel,Park,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center
9,Scarborough,1,College Stadium,General Entertainment,Skating Rink,Café,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
10,Scarborough,1,Indian Restaurant,Chinese Restaurant,Pet Store,Light Rail Station,Gaming Cafe,Vietnamese Restaurant,Breakfast Spot,Bubble Tea Shop,Bakery,Lounge
11,Scarborough,1,Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Bakery,Smoke Shop,Sandwich Place,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station
12,Scarborough,1,Latin American Restaurant,Lounge,Breakfast Spot,Chinese Restaurant,American Restaurant,Pet Store,Korean BBQ Restaurant,Light Rail Station,Medical Center,Metro Station
13,Scarborough,1,Pizza Place,Italian Restaurant,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Chinese Restaurant,Gas Station,Rental Car Location,Noodle House,Bank


### Cluster 3

In [81]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant


### Cluster 4


In [82]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3,Playground,Convenience Store,American Restaurant,Park,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station
14,Scarborough,3,Intersection,Park,Playground,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station


### Cluster 5

In [83]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4,Coffee Shop,Korean BBQ Restaurant,Pet Store,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
